In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os #For OS related functions

In [2]:
# Tensorflow / Keras

from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for assembling a Neural Network model
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, ReLU, LeakyReLU, Dropout # adding layers to the Neural Network model
from tensorflow.keras.utils import plot_model # for plotting model diagram
from tensorflow.keras.optimizers import Adam # for model optimization 


# Data manipulation

import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for scaling inputs used in the generator and discriminator


# Image processing

import cv2 # for ingesting images
print('OpenCV: %s' % cv2.__version__) # print version

#Visualization

import matplotlib 
import matplotlib.pyplot as plt # or data visualizationa
print('matplotlib: %s' % matplotlib.__version__) # print version
import graphviz # for showing model diagram
print('graphviz: %s' % graphviz.__version__) # print version

Tensorflow/Keras: 2.12.0
numpy: 1.23.5
sklearn: 1.2.1
OpenCV: 4.7.0
matplotlib: 3.7.0
graphviz: 0.20.1


In [3]:
Tumor_Label = [] 

# Create a list to store image paths and populate Tumor Label Variable

ImagePaths=[]
for tumor_status in ["glioma_tumor","meningioma_tumor","no_tumor","pituitary_tumor"]:
    count_tumor_status = 0
    ImgLocation = "Training/"+tumor_status
    for image in list(os.listdir(ImgLocation)):
        ImagePaths=ImagePaths+[ImgLocation+"/"+image]
        count_tumor_status += 1
        if tumor_status == "no_tumor":
            Tumor_Label.append("no")
        else:
            Tumor_Label.append("yes")
        
        
# Load images and resize to 256 x 256
data_lowres=[]
for img in ImagePaths:
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_lowres = cv2.resize(image, (256, 256))
    data_lowres.append(image_lowres)
    

    
# Convert image data to numpy array and standardize values (divide by 255 since RGB values ranges from 0 to 255)
data_lowres = np.array(data_lowres, dtype="float") / 255.0
# Show data shape
print("Shape of data_lowres: ", data_lowres.shape)

Shape of data_lowres:  (2870, 256, 256)


In [4]:
Tumor_Label = pd.Series(Tumor_Label,name='Tumor_Status')
Tumor_Label = pd.get_dummies(Tumor_Label)
data_lowres = data_lowres.reshape(-1,256, 256,1)
print(data_lowres.shape)
print(Tumor_Label.shape)

(2870, 256, 256, 1)
(2870, 2)


In [5]:
Tumor_Label_test = [] 

# Create a list to store image paths and populate Tumor Label Variable

ImagePaths_test=[]
for tumor_status in ["glioma_tumor","meningioma_tumor","no_tumor","pituitary_tumor"]:
    count_tumor_status = 0
    ImgLocation = "Testing/"+tumor_status
    for image in list(os.listdir(ImgLocation)):
        ImagePaths_test=ImagePaths_test+[ImgLocation+"/"+image]
        count_tumor_status += 1
        if tumor_status == "no_tumor":
            Tumor_Label_test.append("no")
        else:
            Tumor_Label_test.append("yes")
        
        
# Load images and resize to 256 x 256
data_lowres_test=[]
for img in ImagePaths_test:
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_lowres = cv2.resize(image, (256, 256))
    data_lowres_test.append(image_lowres)
    
    
# Convert image data to numpy array and standardize values (divide by 255 since RGB values ranges from 0 to 255)
data_lowres_test = np.array(data_lowres_test, dtype="float") / 255.0
# Show data shape
print("Shape of data_lowres_test: ", data_lowres_test.shape)

Shape of data_lowres_test:  (394, 256, 256)


In [6]:
Tumor_Label_test = pd.Series(Tumor_Label_test,name='Tumor_Status')
Tumor_Label_test = pd.get_dummies(Tumor_Label_test)
data_lowres_test = data_lowres_test.reshape(-1,256, 256,1)
print(data_lowres_test.shape)
print(Tumor_Label_test.shape)

(394, 256, 256, 1)
(394, 2)


In [7]:
from sklearn.model_selection import StratifiedKFold

# Convert labels from one-hot to integers for StratifiedKFold
y_integers = np.argmax(Tumor_Label.values, axis=1)

# Create a StratifiedKFold object
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# Use it to split your data and labels
splits = skf.split(data_lowres, y_integers)

# `splits` is now a generator of (train_index, test_index) pairs
data_segments = []
label_segments = []
data_segments_test = []
label_segments_test = []
for train_index, test_index in splits:
    X_train, X_test = data_lowres[train_index], data_lowres[test_index]
    y_train, y_test = Tumor_Label.values[train_index], Tumor_Label.values[test_index]
    
    data_segments.append(X_train)
    label_segments.append(y_train)
    data_segments_test.append(X_test)
    label_segments_test.append(y_test)
    
print(len(data_segments_test))
print(data_segments_test[0].shape)
print(len(label_segments_test))
print(label_segments_test[0].shape)

print(len(data_segments))
print(data_segments[0].shape)
print(len(label_segments))
print(label_segments[0].shape)

4
(718, 256, 256, 1)
4
(718, 2)
4
(2152, 256, 256, 1)
4
(2152, 2)


In [8]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.losses import BinaryCrossentropy, KLDivergence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

#Load techer model
teacher_model = load_model('my_model.h5')

# Define student model, which is a smaller version of the teacher model
student_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 1)),
    MaxPooling2D(),
    Dropout(0.1),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(),
    Dropout(0.1),
    BatchNormalization(),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.1),
    BatchNormalization(),

    Dense(2, activation='softmax'),  # Using softmax activation function for distillation
])

In [9]:
import tensorflow as tf
from tensorflow.keras import Model

# Create a teacher-student model
class Distiller(Model):
    def __init__(self, teacher, student):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=10,
    ):
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results


In [11]:
# Now you need to compile and train the Distiller model
distiller = Distiller(teacher=teacher_model, student=student_model)
distiller.compile(
    optimizer='adam',
    metrics=['accuracy'],
    student_loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)
print(Tumor_Label.shape)
print(data_lowres.shape)

distiller.fit(data_lowres, Tumor_Label,batch_size=16, epochs=3)

(2870, 2)
(2870, 256, 256, 1)
Epoch 1/3


C:\Users\Navonil\AppData\Roaming\Python\Python39\site-packages\keras\backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


  5/180 [..............................] - ETA: 10:38 - accuracy: 0.6500 - student_loss: 1.2533 - distillation_loss: 0.0014

KeyboardInterrupt: 

In [24]:
import numpy as np
import tensorflow as tf

# Assuming you have a pre-trained model named "cnn_model"
cnn_model = load_model('my_model.h5') # Load your pre-trained model here

# Here, we define a "client" as a simple object that has a local model and a local dataset
class Client:
    def __init__(self, model, data, labels):
        self.model = model
        self.data = data
        self.labels = labels
        self.model.compile(optimizer='adam', 
                           loss='categorical_crossentropy', 
                           metrics=['accuracy'])

    def train(self):
        self.model.fit(self.data, self.labels, epochs=3, verbose=0)

    def get_weights(self):
        return self.model.get_weights()

    def set_weights(self, weights):
        self.model.set_weights(weights)
    
    def evaluate(self):
        loss, accuracy = self.model.evaluate(self.data, self.labels, verbose=0)
        return loss, accuracy


In [25]:
# We'll simulate federated learning with a simple loop that does the following:
# 1. Sends the current global model to each client
# 2. Each client trains this model on their local data
# 3. Each client sends their trained model back to the server
# 4. The server averages all the models to create a new global model

class FederatedLearning:
    def __init__(self, model, clients):
        self.global_model = model
        self.clients = clients

    def aggregate_weights(self, weights):
        # Aggregate weights from all clients
        return np.mean(weights, axis=0)

    def send_model(self, client):
        # Send model to a client
        client.set_weights(self.global_model.get_weights())

    def receive_model(self, client):
        # Receive model from a client
        return client.get_weights()

    def train(self, rounds=1):
        # Train for a certain number of rounds
        for _ in range(rounds):
            weights = []
            for client in self.clients:
                self.send_model(client)
                client.train()
                weights.append(self.receive_model(client))
                loss, accuracy = client.evaluate()  # Get the loss and accuracy of the client's model
                print(f"Client {self.clients.index(client)} loss: {loss}, accuracy: {accuracy}")
            self.global_model.set_weights(self.aggregate_weights(weights))

# Assume that the dataset (data_lowres and Tumor_Label) is distributed across three clients
client_datasets = [
    (data_lowres[:950], Tumor_Label[:950]),
    (data_lowres[950:1900], Tumor_Label[950:1900]),
    (data_lowres[1900:], Tumor_Label[1900:])
]

# Create a list of clients. Each client gets a copy of the pre-trained model and a slice of the data
clients = [Client(tf.keras.models.clone_model(cnn_model), data, labels) for data, labels in client_datasets]
print(len(clients))
#print(clients)
# Create a federated learning object and run training
fl = FederatedLearning(cnn_model, clients)
fl.train(rounds=5)


3


KeyboardInterrupt: 